#Reference: https://www.datasklr.com/logistic-regression/multinomial-logistic-regression

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
from numpy import nan
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
import pandas as pd
import numpy as np
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import csv
from nltk.tokenize import sent_tokenize
import nltk
!pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
path = "/content/drive/MyDrive/MWP(Words and Values in List).csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df = df.iloc[:,1:3]
df

,Problem,Level
0,There are constants alpha and beta suc...,Level 5
1,The length of the segment between the points ...,Level 5
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5
3,"For some constants a and b, let [f(x) = ...",Level 5
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2
...,...,...
1739,What is the slope of the line passing through ...,Level 2
1740,Evaluate i^{11} + i^{16} + i^{21} + i^{26} + ...,Level 4
1741,If a is a constant such that 4x^2 - 12x + a...,Level 3
1742,"Given a star b = a^2 + 2ab + b^2 , what is ...",Level 3


In [6]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)
df.head()

,Problem,Level
0,There are constants alpha and beta suc...,5
1,The length of the segment between the points ...,5
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5
3,"For some constants a and b, let [f(x) = ...",5
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2


#Function to count number of sentences in the problem.

In [7]:
def count_sentences(data):
  number_of_sentences = sent_tokenize(data)
  return len(number_of_sentences)

#Count Words

In [8]:
def count_words(data):
  number_of_words = len(data.split())
  return number_of_words

#Math Words

In [9]:
path = "/content/drive/MyDrive/Math Words.csv"
df_cw = pd.read_csv(path)
df_cw = df_cw.iloc[:,1]
df_cw.head()

0       x
1    What
2    frac
3       f
4       b
Name: Word, dtype: object

In [10]:
#Say we choose words of length >2 alone - to avoid x, f, b etc. 
complex_words = []
for i in df_cw:
  if len(i)>1:
    complex_words.append(i)

In [11]:
def count_cw(data):
  total_count = 0
  data = data.split()
  for word in data: 
    if word in complex_words:
      total_count = total_count+1
  return total_count

#Math Symbols

In [12]:
math_symbols = []
l = "!@#$%^&*()[]{}:;<>+=-/?.,"
for i in l:
  math_symbols.append(i)

In [13]:
def count_symbols(data):
  total_count = 0
  data = data.split()
  for word in data: 
    if word in math_symbols:
      total_count = total_count+1
  return total_count

#Count of Numbers 

In [14]:
def count_nums(data):
  total_count = 0
  data = data.split()
  for word in data: 
    if word.isalpha() == False:
      total_count = total_count+1
  return total_count

#Extract Features

In [15]:
WordsPerSentence = []
MathWordsPerSentence = []
MathSymbolsPerSentence = [] 
NumCountPerSentence = []

In [16]:
for i in df.iloc[:,0]:
  s = count_sentences(i)
  w = count_words(i)
  mw = count_cw(i)
  ms = count_symbols(i)
  n = count_nums(i) 

  WordsPerSentence.append(w/s) 
  MathWordsPerSentence.append(mw/s)  
  MathSymbolsPerSentence.append(ms/s)   
  NumCountPerSentence.append(n/s)  

In [17]:
df.head()

,Problem,Level
0,There are constants alpha and beta suc...,5
1,The length of the segment between the points ...,5
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5
3,"For some constants a and b, let [f(x) = ...",5
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2


In [18]:
df_data = pd.DataFrame() 
df_data["WordsPerSentence"] = WordsPerSentence
df_data["MathWordsPerSentence"] = MathWordsPerSentence
df_data["MathSymbolsPerSentence"] = MathSymbolsPerSentence
df_data["NumCountPerSentence"] = NumCountPerSentence
df_data["Level"] = df.iloc[:,1]
df_data.head()

,WordsPerSentence,MathWordsPerSentence,MathSymbolsPerSentence,NumCountPerSentence,Level
0,9.500000,3.0,1.500000,4.000000,5
1,14.000000,4.0,0.500000,4.000000,5
2,16.200000,7.2,0.200000,2.000000,5
3,16.666667,3.0,3.666667,8.666667,5
4,15.000000,3.0,6.000000,11.000000,2


#Logistic Regression

In [19]:
X = df_data.drop(['Level'], axis=1) 
y = df_data['Level']

In [20]:
#Combine Difficulty Levels: 1,2=1. 3=2. 4,5=3.
y = y.replace(to_replace = 2, value = 1)
y = y.replace(to_replace = 3, value = 2)
y = y.replace(to_replace = 4, value = 3)
y = y.replace(to_replace = 5, value = 3)

In [21]:
y

0       3
1       3
2       3
3       3
4       1
       ..
1739    1
1740    3
1741    2
1742    2
1743    3
Name: Level, Length: 1744, dtype: int64

In [22]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 4)
(349, 4)
(1395,)
(349,)


In [23]:
model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='l1', solver='saga').fit(X_train, y_train)
preds = model1.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [24]:
params = model1.get_params()
print(params)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'l1', 'random_state': 0, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [25]:
print('Intercept: \n', model1.intercept_)
print('Coefficients: \n', model1.coef_)

Intercept: 
 [ 0.20013719 -0.11813336 -0.08200384]
Coefficients: 
 [[-3.96370053e-02  2.50929101e-02  4.01972355e-02  2.67869156e-02]
 [-1.05489045e-02  0.00000000e+00  1.84018422e-02 -2.65519501e-07]
 [ 6.10977211e-02 -1.87787316e-02 -6.94124857e-02 -2.41794554e-02]]


In [26]:
np.exp(model1.coef_)

array([[0.96113826, 1.02541039, 1.04101608, 1.02714891],
       [0.98950654, 1.        , 1.0185722 , 0.99999973],
       [1.06300279, 0.98139649, 0.93294178, 0.97611053]])

In [27]:
print('Accuracy Score:', metrics.accuracy_score(y_test, preds))  

Accuracy Score: 0.5243553008595988


In [37]:
y_train = y_train.tolist()
type(y_train)

list

In [51]:
#One hot encoding of Y_train, Y_test
b = np.zeros((1395, 3)) #Train data size=1394
j = 0
for j in range(1395):
  b[j, y_train[j]-1]=1 #-1 to account for 0 based index

In [59]:
#Initialising ANN - 2 hidden layers, 1 input layer, and 1 output layer
ann = Sequential([

#Adding First Hidden Layer
Dense(units=6,activation="relu", input_shape = (X_train.shape[1],)), #6=number of neurons in 1st hidden layer. 
#Adding Hidden Layers
Dense(units=12,activation="relu"),
Dense(units=24,activation="relu"),
Dense(units=48,activation="relu"),
Dense(units=96,activation="relu"),

#Adding Output Layer
#For the binary classification Problems, the activation function that should always be used is sigmoid.
#For a multiclass classification problem, the activation function that should be used is softmax.
Dense(units=3,activation="softmax"), #3 neurons since 3 classes. 

]) 



In [60]:
#Compiling ANN
ann.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
#Fitting ANN
hist = ann.fit(X_train, b, batch_size=32, epochs = 100) 

Epoch 1/100
44/44 [==============================] - 1s 2ms/step - loss: 1.0387 - accuracy: 0.4753
Epoch 2/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0321 - accuracy: 0.5018
Epoch 3/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0256 - accuracy: 0.5054
Epoch 4/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0259 - accuracy: 0.4996
Epoch 5/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0222 - accuracy: 0.5068
Epoch 6/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0194 - accuracy: 0.5111
Epoch 7/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0144 - accuracy: 0.5219
Epoch 8/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0153 - accuracy: 0.5111
Epoch 9/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0132 - accuracy: 0.5125
Epoch 10/100
44/44 [==============================] - 0s 2ms/step - loss: 1.0220 - accuracy: 0.5004
Epoch 11/